In [8]:
from collections import Counter
import numpy as np

### HELPERS

In [39]:
def election(rows, threshold):
    
    # VOTE & THRESHOLD FOR DECISION
    votes = []
    
    # ROW DIMENSIONS
    dimensions = len(rows.shape)
    
    # LOOP THROUGH EACH ELEMENT
    for row in rows:
        
        # MULTI-DIMENSIONAL (FEATURES)
        if dimensions == 2:
            
            # SUBCONTAINER FOR COLUMN
            sub_collector = []
            
            # EVALUATE WHOLE COLUMN
            for value in row:
                vote = decide_vote(value, threshold)
                sub_collector.append(vote)
            
            # COUNT THE VOTES
            count = Counter(list(sub_collector))
            
            # FIND THE POPULAR VOTE
            popular = count.most_common()[0][0]
            
            # APPEND IT
            votes.append(popular)
        
        # ONE-DIMENSIONAL (LABELS)
        else:
            vote = decide_vote(row, threshold)
            votes.append(vote)
            
    return votes

In [37]:
def decide_vote(value, threshold):
    if value > threshold:
        return 'buy'
    elif value < -threshold:
        return 'sell'
    else:
        return 'hold'

In [38]:
def rename_labels(values, names):
    container = []
    for value in values:
        numeric = names[value]
        container.append(numeric)
    return container

### TRIGGER

In [40]:
def classify(params):
    
    # DECONSTRUCT PARAMS
    original = params['predictions']
    label_names = params['label_names']
    threshold = params['threshold']
    
    # CLONE THE ORIGINAL
    dataframe = original.copy()
    
    # LOOP THROUGH EACH COLUMN
    for column in dataframe.columns:
        
        # CALCULATE LOG VALUE BY CURRENT & NEXT
        dataframe[column] = np.log(dataframe[column] / dataframe[column].shift(1))
        
    # DROP ROWS WITH NAN VALUES
    dataframe.dropna(inplace=True)
    
    # EXTRACT FEATUERS & LABELS
    labels = dataframe['label'].to_numpy()
    features = dataframe.loc[:, dataframe.columns != 'label'].to_numpy()
    
    # EXTRACT BUY/SELL/HOLD VOTES FROM FEATURES & LABELS
    feature_votes = election(features, threshold)
    label_votes = election(labels, threshold)
    
    # ADD VOTING COLUMNS
    dataframe['buy'] = 0
    dataframe['sell'] = 0
    dataframe['hold'] = 0
    
    # LOOP THROUGH DATAFRAME ROWS
    for index, (date, row) in enumerate(dataframe.iterrows()):

        # QUERY ROW VOTE & INJECT IT INTO MATRIX
        choice = feature_votes[index]
        dataframe.at[date, choice] = 1
    
    # CHANGE LABEL STRINGS TO NUMERIC REPRESENTATIONS
    #label_votes = rename_labels(label_votes, label_names)
    
    # REPLACE THE OLD LABELS
    dataframe['label'] = label_votes
    
    return dataframe